In [5]:
pip install beyond

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
from beyond.io.tle import Tle
from beyond.frames import create_station
from beyond.dates import Date, timedelta

# Parse TLE
tle = Tle("""NOAA 15 [B]             
1 25338U 98030A   20205.85092036  .00000065  00000-0  45627-4 0  9992
2 25338  98.7149 230.7131 0010993 151.5401 208.6381 14.25980908154168""")

# Create a station from which to compute the pass
station = create_station('Norwich', (52, 1, 0))

counter = 0

for orb in station.visibility(tle.orbit(), start=Date.now(), stop=timedelta(days=2), step=timedelta(minutes=2), events=True):

    # As all angles are given in radians,
    # there is some conversion to do
    azim = -np.degrees(orb.theta) % 360
    elev = np.degrees(orb.phi)
    r = orb.r / 1000.

    print("{event:10} {tle.name}  {date:%Y-%m-%dT%H:%M:%S.%f} {azim:7.2f} {elev:7.2f} {r:10.2f}".format(
        date=orb.date, r=r, azim=azim, elev=elev,
        tle=tle, event=orb.event if orb.event is not None else ""
    ))

    # Stop at the end of the first pass
    if orb.event and orb.event.info == "LOS":
        counter += 1
        if counter >= 5:
            break




AOS 0 Norwich NOAA 15 [B]  2025-07-21T19:13:09.770135  157.02    0.00    3318.31
           NOAA 15 [B]  2025-07-21T19:13:21.720597  156.96    0.73    3238.56
           NOAA 15 [B]  2025-07-21T19:15:21.720597  155.97    9.40    2442.08
           NOAA 15 [B]  2025-07-21T19:17:21.720597  153.58   22.78    1675.31
           NOAA 15 [B]  2025-07-21T19:19:21.720597  144.03   49.69    1032.70
MAX Norwich NOAA 15 [B]  2025-07-21T19:20:46.536155   71.34   76.46     840.59
           NOAA 15 [B]  2025-07-21T19:21:21.720597   18.02   67.89     877.42
           NOAA 15 [B]  2025-07-21T19:23:21.720597  351.36   31.52    1385.82
           NOAA 15 [B]  2025-07-21T19:25:21.720597  347.61   14.31    2121.10
           NOAA 15 [B]  2025-07-21T19:27:21.720597  346.36    4.23    2909.24
LOS 0 Norwich NOAA 15 [B]  2025-07-21T19:28:27.680103  346.05   -0.00    3348.27
AOS 0 Norwich NOAA 15 [B]  2025-07-21T20:54:21.419882  207.72    0.00    3318.99
           NOAA 15 [B]  2025-07-21T20:55:21.720597  21

In [7]:
import matplotlib.pyplot as plt

In [8]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path

from beyond.io.tle import Tle
from beyond.dates import Date, timedelta


# Parsing of TLE
tle = Tle("""NOAA 15 [B]             
1 25338U 98030A   20191.81750536  .00000057  00000-0  42639-4 0  9999
2 25338  98.7153 216.7534 0010156 191.4103 168.6848 14.25977849152168""")

# Create a station from which to compute the pass
# Conversion into `Orbit` object
orb = tle.orbit()

# Tables containing the positions of the ground track
latitudes, longitudes = [], []
prev_lon, prev_lat = None, None

period = orb.infos.period

#start = orb.date - period
start = Date.now()
stop = 1 * period
step = period / 100

for point in orb.ephemeris(start=start, stop=stop, step=step):

    # Conversion to earth rotating frame
    point.frame = 'ITRF'

    # Conversion from cartesian to spherical coordinates (range, latitude, longitude)
    point.form = 'spherical'

    # Conversion from radians to degrees
    lon, lat = np.degrees(point[1:3])

    # Creation of multiple segments in order to not have a ground track
    # doing impossible paths
    if prev_lon is None:
        lons = []
        lats = []
        longitudes.append(lons)
        latitudes.append(lats)
    elif orb.i < np.pi /2 and (np.sign(prev_lon) == 1 and np.sign(lon) == -1):
        lons.append(lon + 360)
        lats.append(lat)
        lons = [prev_lon - 360]
        lats = [prev_lat]
        longitudes.append(lons)
        latitudes.append(lats)
    elif orb.i > np.pi/2 and (np.sign(prev_lon) == -1 and np.sign(lon) == 1):
        lons.append(lon - 360)
        lats.append(lat)
        lons = [prev_lon + 360]
        lats = [prev_lat]
        longitudes.append(lons)
        latitudes.append(lats)

    lons.append(lon)
    lats.append(lat)
    prev_lon = lon
    prev_lat = lat

img = "earth2.png"
#with cbook.get_sample_data('earth2.png') as img:
#    im = plt.imread(img)

im = plt.imread(img)
plt.figure(figsize=(15.2, 8.2))
plt.imshow(im, extent=[-180, 180, -90, 90])

for lons, lats in zip(longitudes, latitudes):
    plt.plot(lons, lats, 'w', linestyle=":", linewidth=3)

torb1 = orb.copy(frame='ITRF', form='spherical').propagate(start)
lon, lat = np.degrees(torb1[1:3])
#torb = np.degrees(orb.ephemeris(start=start)[0][1:3])
#lon, lat = np.degrees(orb.copy(frame='ITRF', form='spherical')[1:3])
plt.plot([lon], [lat], 'ro')

#lon, lat = np.degrees(orb.copy(frame='ITRF', form='spherical')[1:3])
#plt.plot([lon], [lat], 'ro')
dir(plt)

#plt.xlim([-20, 20])
#plt.ylim([30, 70])
plt.xlim([-180, 180])
plt.ylim([-90, 90])

plt.grid(True, color='w', linestyle=":", alpha=0.4)
#plt.xticks(range(-20, 20, 2))
#plt.yticks(range(30, 70, 2))
plt.xticks(range(-180, 181, 30))
plt.yticks(range(-90, 91, 30))
plt.tight_layout()

#print(plt.type())
print(dir(plt.axis()))

#plt.axes['bottom'].set_color('#dddddd')
#plt.Axes.spines['top'].set_color('#dddddd') 
#plt.Axes.spines['right'].set_color('red')
#plt.Axes.spines['left'].set_color('red')

#plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'earth2.png'

In [ ]:
torb1 = orb.propagate(start).copy(frame='ITRF', form='spherical')
lon, lat = np.degrees(torb1[1:3])
print("%s %s" % (lon, lat))

lon, lat = np.degrees(orb.copy(frame='ITRF', form='spherical')[1:3])
print("%s %s" % (lon, lat))


In [ ]:
orb.date

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

from beyond.dates import Date, timedelta
from beyond.io.tle import Tle
from beyond.frames import create_station
from beyond.config import config


tle = Tle("""NOAA 15 [B]             
1 25338U 98030A   20205.85092036  .00000065  00000-0  45627-4 0  9992
2 25338  98.7149 230.7131 0010993 151.5401 208.6381 14.25980908154168""")

# Create a station from which to compute the pass
station = create_station('Norwich', (52.6, 1.19, 0))
azims, elevs = [], []

orbit = tle.orbit()

start = None
end = None
max = max

for orb in station.visibility(orbit, start=Date.now(), stop=timedelta(hours=24), step=timedelta(seconds=30), events=True):
    elev = np.degrees(orb.phi)
    # Radians are counterclockwise and azimuth is clockwise
    azim = np.degrees(-orb.theta) % 360

    # Archive for plotting
    azims.append(azim)
    # Matplotlib actually force 0 to be at the center of the polar plot,
    # so we trick it by inverting the values
    elevs.append(90 - elev)

    r = orb.r / 1000.
    
    if orb.event is not None:
        if orb.event.info =='AOS':
            start = orb.date
        if orb.event.info  == 'MAX':
            max = elev
        
        if orb.event.info  == 'LOS':
            
            end = orb.date
            if max >= 10:
                print("Pass: %s %s %s" % (start, max, end))
    
    #print("{event:7} {orb.date:%H:%M:%S} {azim:7.2f} {elev:7.2f} {r:10.2f} {orb.r_dot:10.2f}".format(
    #    orb=orb, r=r, azim=azim, elev=elev, event=orb.event.info if orb.event is not None else ""
    #))
    
    #print (orb.event.info)

    #if orb.event and orb.event.info.startswith("LOS"):
    #    # We stop at the end of the first pass
    #    print()
    #    break

plt.figure()
ax = plt.subplot(111, projection='polar')
ax.set_theta_direction(-1)
ax.set_theta_zero_location('N')
plt.plot(np.radians(azims), elevs, '.')
ax.set_yticks(range(0, 90, 20))
ax.set_yticklabels(map(str, range(90, 0, -20)))
ax.set_rmax(90)


plt.show()


# Predictions

In [9]:
import requests
import numpy as np
from beyond.dates import Date, timedelta
from beyond.io.tle import Tle
from beyond.frames import create_station
from beyond.config import config


# In[3]:


celestrak_noaa =  "https://celestrak.org/NORAD/elements/gp.php?GROUP=noaa&FORMAT=tle"

data = requests.get(celestrak_noaa)
data = data.content.decode("utf-8")
rows = data.splitlines()

tles = {}
first = None
second = None
last = None
for row in rows:
    if not first:
        first = row
    elif not second:
        second = row
    else:
        last = row
        name = (first.split("[")[0]).strip()
        tles[name] = """%s
%s
%s""" % (first, second, last)
        first = None
        second= None
        



In [10]:

tles['NOAA 15']


'NOAA 15                 \n1 25338U 98030A   25202.53139303  .00000185  00000+0  93588-4 0  9999\n2 25338  98.5339 226.6500 0009268 232.2392 127.7950 14.26999468414388'

In [11]:

tle = Tle(tles['NOAA 15'])


In [12]:

station = create_station('Norwich', (52.5, 1.2, 0)) # obfuscated slightly


A frame with the name 'Norwich' is already registered. Overriding


In [13]:

def predictions(tle, station, hours):
    azims, elevs = [], []

    orbit = tle.orbit()

    start = None
    end = None
    max_alt = None

    for orb in station.visibility(orbit, start=Date.now(), stop=timedelta(hours=hours), step=timedelta(seconds=30), events=True):
        elev = np.degrees(orb.phi)
        # Radians are counterclockwise and azimuth is clockwise
        azim = np.degrees(-orb.theta) % 360

        # Archive for plotting
        azims.append(azim)
        # Matplotlib actually force 0 to be at the center of the polar plot,
        # so we trick it by inverting the values
        elevs.append(90 - elev)

        #r = orb.r / 1000.

        if orb.event is not None:
            if orb.event.info =='AOS': # aquisition of signal
                start = orb.date
            if orb.event.info  == 'MAX': 
                max_alt = elev

            if orb.event.info  == 'LOS': # loss of signal

                end = orb.date
                if max_alt and max_alt >= 10:
                    print("Pass: %s %s %s" % (start, max_alt, end))
                    start = None
                    end = None
                    max_alt = None

In [17]:

predictions(Tle(tles['NOAA 19']), station, 24)


Pass: 2025-07-21T19:49:52.698037 UTC 35.03473447796783 2025-07-21T20:04:53.256058 UTC
Pass: 2025-07-21T21:30:03.877502 UTC 48.91371093414219 2025-07-21T21:45:42.551027 UTC
Pass: 2025-07-22T09:45:11.592997 UTC 46.31613877178635 2025-07-22T10:00:43.641130 UTC
Pass: 2025-07-22T11:25:59.732954 UTC 36.43051235065689 2025-07-22T11:41:01.232550 UTC


In [15]:

predictions(Tle(tles['NOAA 18']), station, 24)


Pass: 2025-07-21T19:34:09.030736 UTC 12.42685256703339 2025-07-21T19:46:08.294105 UTC
Pass: 2025-07-21T21:11:57.768688 UTC 50.374592099931185 2025-07-21T21:27:17.052698 UTC
Pass: 2025-07-21T22:53:01.778404 UTC 31.719460832059937 2025-07-21T23:07:56.578660 UTC
Pass: 2025-07-22T09:27:18.841343 UTC 13.469435677747422 2025-07-22T09:40:06.538969 UTC
Pass: 2025-07-22T11:07:25.604256 UTC 72.46074875059531 2025-07-22T11:23:13.148462 UTC
Pass: 2025-07-22T12:48:22.708637 UTC 25.07877802595632 2025-07-22T13:02:37.352724 UTC


In [16]:

predictions(Tle(tles['NOAA 15']), station, 24)


Pass: 2025-07-21T19:36:36.610221 UTC 21.768342111457635 2025-07-21T19:50:27.017908 UTC
Pass: 2025-07-22T06:04:18.441467 UTC 17.39444568349942 2025-07-22T06:17:30.237092 UTC
Pass: 2025-07-22T07:43:43.498350 UTC 86.18708806416475 2025-07-22T07:58:58.254121 UTC
Pass: 2025-07-22T09:23:52.459682 UTC 18.87295519525284 2025-07-22T09:36:56.964218 UTC
Pass: 2025-07-22T15:53:02.170167 UTC 10.99806632855495 2025-07-22T16:04:22.172635 UTC
Pass: 2025-07-22T17:29:47.913088 UTC 45.90938030275255 2025-07-22T17:44:41.318743 UTC
